In [2]:
# https://medium.com/@aminajavaid30/building-a-rag-system-synthesis-67f36efa7c35

# Data Ingestion & Retrieval
import bs4
import re
import os
import getpass
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.documents import Document
from langchain_community.document_loaders import RecursiveUrlLoader
from langchain_community.document_loaders import GithubFileLoader
from bs4 import BeautifulSoup



In [3]:
# Load a github repo

if not os.environ.get("GITHUB_PAT"):
  os.environ["GITHUB_PAT"] = getpass.getpass("Enter Github PAT: ")

async def github_repo(repo_name: str) -> list[Document]:
    loader = GithubFileLoader(
        repo=repo_name, # the repo name
        access_token= os.environ["GITHUB_PAT"],
        github_api_url="https://api.github.com",
        branch ="master", 
        file_filter=lambda file_path: file_path.endswith(
            ".md"
        ),  # load all markdowns files.
    )
    github_docs = []
    async for doc in loader.alazy_load():
        github_docs.append(doc)
    return github_docs

In [4]:
# Load a website reccusivly and split it into chunks
def bs4_extractor(html: str) -> str:
    soup = BeautifulSoup(html, "lxml")
    return re.sub(r"\n\n+", "\n\n", soup.text).strip()

async def website_parser(base_url: str) -> list[Document]:
    loader = RecursiveUrlLoader(
        base_url,
        extractor=bs4_extractor,
        max_depth=2,
        prevent_outside=True,
        # use_async=False,
        # extractor=None,
        # metadata_extractor=None,
        # exclude_dirs=(),
        # timeout=10,
        # check_response_status=True,
        # continue_on_failure=True,
        # 
        # base_url=None,
        # ...
    )
    site_docs = []
    async for doc in loader.alazy_load():
        site_docs.append(doc)
    return site_docs

In [5]:
# Initialize the OpenAI embedding model
import getpass
import os

from langchain_openai import AzureOpenAIEmbeddings
from langchain_core.vectorstores import InMemoryVectorStore

if not os.environ.get("AZURE_OPENAI_API_KEY"):
  os.environ["AZURE_OPENAI_API_KEY"] = getpass.getpass("Enter API key for Azure: ")

azureEmbeddings = AzureOpenAIEmbeddings(
    azure_endpoint="https://oai-bim42-test-fr-ai.openai.azure.com",
    azure_deployment="text-embedding-3-small",
    openai_api_version="2025-01-01-preview",
)

# Initialize the recursive character text splitter
text_splitter = RecursiveCharacterTextSplitter(              
    chunk_size=1000,
    chunk_overlap=200,
    add_start_index=True
)

# Initialize with an embedding model
vector_store = InMemoryVectorStore(embedding=azureEmbeddings)

In [6]:
# Define a list of site to parse
urls = ["https://xeokit.io/","https://xeokit.github.io/xeokit-bim-viewer/"]

# Iterate over each urls in the list
for url in urls:
    # Retrive the documents
    website_docs = await website_parser(url)
    website_docs
    print(f"{len(website_docs)} docs in {url}")
    # Split the documents into chunks
    chunks = text_splitter.split_documents(website_docs)
    print(f"{len(chunks)} chunks in {url}")
    # Add embeddings into the vector store
    vector_store.add_documents(documents=chunks)


17 docs in https://xeokit.io/
86 chunks in https://xeokit.io/
15 docs in https://xeokit.github.io/xeokit-bim-viewer/
271 chunks in https://xeokit.github.io/xeokit-bim-viewer/


In [7]:
# Define a list of github repo to parse
repos = ["xeokit/xeokit-sdk","xeokit/xeokit-bim-viewer"]


# Iterate over each urls in the list
for repo in repos:
    # Retrive the documents
    github_docs = await github_repo(repo)
    print(f"{len(github_docs)} docs in {repo}")
    # Split the documents into chunks
    chunks = text_splitter.split_documents(github_docs)
    print(f"{len(chunks)} chunks in {repo}")
    # Add embeddings into the vector store
    vector_store.add_documents(documents=chunks)

14 docs in xeokit/xeokit-sdk
196 chunks in xeokit/xeokit-sdk
4 docs in xeokit/xeokit-bim-viewer
107 chunks in xeokit/xeokit-bim-viewer


In [8]:
# Retrieve relevant information using similarity search
retriever = vector_store.as_retriever() # uses similarity search by default

In [9]:
# Q/A Retrieval Chain

from langchain_openai import AzureChatOpenAI
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

# Initialize the Azure AI LLM with the specific model
llm = AzureChatOpenAI(
    azure_endpoint="https://oai-bim42-test-fr-ai.openai.azure.com",
    azure_deployment="gpt-4o-mini",
    openai_api_version="2025-01-01-preview",
)

# Define the system prompt that instructs the LLM how to answer questions based on retrieved context
system_prompt = (
    "You are a specialist of Building Information modeling looking for a web-based building model viewer"    
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer the question."
    "Add sources like links to your answer."
    "If you don't know the answer, say that you don't know."
    "Use four sentences maximum and keep the answer concise."
    "\n\n"
    "{context}"  # Placeholder for the retrieved context
)

# Create a chat prompt template with a system message and human message
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),  # System message contains instructions and context
        ("human", "{input}"),  # Human message includes the user's input question
    ]
)

# Create a document chain that combines the LLM with the prompt
question_answer_chain = create_stuff_documents_chain(llm, prompt)

# Combine the retrieval chain with the question-answering chain 
# The retrieval chain retrieves relevant documents and feeds them into the question-answering chain
retrieval_chain = create_retrieval_chain(retriever, question_answer_chain)

In [11]:
# Invoke the chain with a question, and the retriever will provide context for the LLM to generate an answer
response = retrieval_chain.invoke({"input": "Please describe the software editor behind this solution (name, description, history, "})
print(response['answer'])

The software editor behind the web-based building model viewer mentioned is xeokit. Xeokit is a JavaScript framework that focuses on engineering graphics for the web, specifically tailored for visualizing large-scale 3D models in browsers. It has grown from over 10 years of experience in the field and has become a go-to solution for various applications, including construction and infrastructure projects. Xeokit has also established partnerships, such as with D-Studio, to enhance its features and capabilities further. For more information, you can visit their site: [xeokit](https://xeokit.com).


In [12]:
response = retrieval_chain.invoke({"input": "What is the price or licencing model of the solution?"})
print(response['answer'])

The xeokit SDK is open-source software licensed under the Affero GPL version 3 license, which is free to use. Additionally, they offer custom proprietary licenses for enterprises that include full support with flexible terms. The specific pricing for the custom licenses would need to be discussed directly with the provider, Creoox AG. For more information, you can visit the xeokit website.


In [13]:
response = retrieval_chain.invoke({"input": "Can you list all the input format accepted by the solution?"})
print(response['answer'])

I don't know.


In [14]:
response = retrieval_chain.invoke({"input": "What is the format used in the web viewer ?"})
print(response['answer'])

The web viewer supports IFC formats, specifically IFC2x3 and IFC4. This allows for effective loading and rendering of building information models. For more details, you can visit the [xeokit viewer features section](https://github.com/xeokit/xeokit-bim-viewer/blob/master/app/index.html).


In [16]:
response = retrieval_chain.invoke({"input": "Is there is any mention of BCF?"})
print(response['answer'])

Yes, there is a mention of BCF (BIM Collaborative Format) in the context of managing issues on a BIM project. The BCF records capture the visual state of a BIM viewer, including parameters like camera position and object visibility. You can use methods like `BIMViewer#loadBCFViewpoint()` to load and `BIMViewer#saveBCFViewpoint()` to save BCF records. For more details, you can refer to the documentation at [xeokit](https://xeokit.io).
